In [7]:
import requests
import pandas as pd
import numpy as np


# url = "https://api.opensea.io/api/v2/collections/mutant-ape-yacht-club/stats"
# 
# headers = {
#     "accept": "application/json",
#     "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"
# }
# 
# response = requests.get(url, headers=headers)

url = "https://api.opensea.io/api/v2/collections/mutant-ape-yacht-club"

headers = {
    "accept": "application/json",
    "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"
}

response = requests.get(url, headers=headers)


response_data = response.json()

import pandas as pd

# Assuming response_data is your JSON data
collection_data = {
    "collection": response_data["collection"],
    "name": response_data["name"],
    "description": response_data["description"],
    "image_url": response_data["image_url"],
    "banner_image_url": response_data["banner_image_url"],
    "owner": response_data["owner"],
    "category": response_data["category"],
    "is_disabled": response_data["is_disabled"],
    "is_nsfw": response_data["is_nsfw"],
    "trait_offers_enabled": response_data["trait_offers_enabled"],
    "collection_offers_enabled": response_data["collection_offers_enabled"],
    "opensea_url": response_data["opensea_url"],
    "project_url": response_data["project_url"],
    "wiki_url": response_data["wiki_url"],
    "discord_url": response_data["discord_url"],
    "telegram_url": response_data["telegram_url"],
    "twitter_username": response_data["twitter_username"],
    "instagram_username": response_data["instagram_username"],
    "total_supply": response_data["total_supply"],
    "created_date": response_data["created_date"]
}

df_collection = pd.DataFrame([collection_data])


In [8]:
df_contracts = pd.DataFrame(response_data["contracts"])
df_fees = pd.DataFrame(response_data["fees"])
df_payment_tokens = pd.DataFrame(response_data["payment_tokens"])
df_rarity = pd.DataFrame([response_data["rarity"]])


In [11]:
print(df_collection)
print(df_contracts)
print(df_fees)
print(df_payment_tokens)
print(df_rarity)

              collection                   name  \
0  mutant-ape-yacht-club  Mutant Ape Yacht Club   

                                         description  \
0  The MUTANT APE YACHT CLUB is a collection of u...   

                                           image_url  \
0  https://i.seadn.io/gae/lHexKRMpw-aoSyB1WdFBff5...   

                                    banner_image_url  \
0  https://i.seadn.io/gae/5c-HcdLMinTg3LvEwXYZYC-...   

                                        owner category  is_disabled  is_nsfw  \
0  0xcac7c29bac18f080b98a3076b4e165886f509251     pfps        False    False   

   trait_offers_enabled  collection_offers_enabled  \
0                  True                       True   

                                         opensea_url project_url wiki_url  \
0  https://opensea.io/collection/mutant-ape-yacht...                        

  discord_url telegram_url twitter_username instagram_username  total_supply  \
0                                                    

In [12]:
df_collection.to_csv('collection_info.csv', index=False)
df_contracts.to_csv('collection_contracts.csv', index=False)
df_fees.to_csv('collection_fees.csv', index=False)
df_payment_tokens.to_csv('collection_payment_tokens.csv', index=False)
df_rarity.to_csv('collection_rarity.csv', index=False)
